In [15]:
import numpy as np
import pandas as pd
import multiprocessing as mp
import pickle
import gzip

# Define functions and values to use below

In [12]:
def weightedavg(dfsingle):
    totalobs = np.sum(dfsingle['Num. observations'])
    mult = dfsingle.multiply(dfsingle['Num. observations'],axis=0)
    return mult.sum()/totalobs

In [9]:
# Use local file 
path = '../data/daydatamat.csv'
path = 'daydatamat.csv'

# Serial

In [25]:
%%time
for i in range(10):
    print(i)
    df = pd.read_csv(path,index_col=0)
    dfavg = df.groupby('Bee unique ID').mean()
    df_avg_weighted = df.groupby('Bee unique ID').apply(weightedavg)
    pickle.dump([dfavg,df_avg_weighted],gzip.open('/Users/jacob/Downloads/avgs'+str(i)+'.pklz','wb'))

0
1
2
3
4
5
6
7
8
9
CPU times: user 1min 42s, sys: 1.38 s, total: 1min 44s
Wall time: 1min 43s


# Parallel

In [26]:
mp.cpu_count()

10

In [27]:
%%time
# Function to process each iteration
def process_iteration(i):
    df = pd.read_csv(path, index_col=0)
    dfavg = df.groupby('Bee unique ID').mean()
    df_avg_weighted = df.groupby('Bee unique ID').apply(weightedavg)
    # Save the results using pickle and gzip for compression
    # The result is a list [dfavg, df_avg_weighted] stored in a .pklz file
    pickle.dump([dfavg, df_avg_weighted], gzip.open('/Users/jacob/Downloads/avgs'+str(i)+'.pklz', 'wb'))

# 'get_context' is used here to specify the context for multiprocessing
# This is necessary for compatibility with Mac M1 since it doesn't support 'fork' by default
pool = mp.get_context("fork").Pool(processes=10)

# 'pool.map' applies the 'process_iteration' function to each item in range(10)
# This runs the function in parallel across 10 processes
pool.map(process_iteration, range(10))

# Close the pool after all processes are finished
pool.close()

CPU times: user 6.18 ms, sys: 54.9 ms, total: 61 ms
Wall time: 14 s
